In [1]:
!pip install pmaw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 25.6 MB/s 
     |████████████████████████████████| 55 kB 1.9 MB/s 


In [2]:
from pmaw import PushshiftAPI
import datetime as dt

from time import sleep
import requests
import pandas as pd

In [3]:
# Extração local no subreddit do país

start_time = int(dt.datetime(2020, 1, 1).timestamp())  
end_time = int(dt.datetime(2020, 7, 31).timestamp())   

api = PushshiftAPI()
posts = api.search_submissions(subreddit='argentina',
                               after = start_time, 
                               before = end_time, 
                               q = '"coronavirus"||"covid"||"covid19"||"covid-19"||"corona virus"||"sars cov"||"sarscov"||"sars-cov-2"',
                               limit = None,
                               fields = ['id', 'author', 'created_utc', 'domain', 'url', 'title', 'num_comments', 'full_link', 'permalink'])
post_list_local = [post for post in posts]

df_post = pd.json_normalize(post_list_local)
print(df_post['id'].shape)

comments = []
i = 1
for post in df_post['id'].values:
  r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
  while r.status_code != 200:
    sleep(min((2**i), 60))
    r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
    i += 1

  i = 1
  comments.append(r.json())

pd.json_normalize(pd.json_normalize(comments).explode('data')['data']).to_csv('argentina_local.csv')

(2046,)


In [ ]:
# Extração geral no subreddit Coronavirus

start_time = int(dt.datetime(2020, 1, 1).timestamp())  
end_time = int(dt.datetime(2020, 7, 31).timestamp())   

api = PushshiftAPI()
posts = api.search_submissions(subreddit='Coronavirus',
                               after = start_time, 
                               before = end_time, 
                               q = 'argentina',
                               limit = None,
                               fields = ['id', 'author', 'created_utc', 'domain', 'url', 'title', 'num_comments', 'full_link', 'permalink'])
post_list_global = [post for post in posts]

df_post = pd.json_normalize(post_list_global)
print(df_post['id'].shape)

comments = []
i = 1
for post in df_post['id'].values:
  r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
  while r.status_code != 200:
    sleep(min((2**i), 60))
    r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
    i += 1

  i = 1
  comments.append(r.json())

pd.json_normalize(pd.json_normalize(comments).explode('data')['data']).to_csv('argentina_global.csv')

(275,)
